In [ ]:
import numpy as np
import pickle
import random
import igraph
import networkx as nx

In [ ]:
# Set random seed
RANDOM_SEED = 1234
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [ ]:
verbose = True
# Generate test graph instances
for num_edges in [500]:
    num_nodes = int(np.floor(num_edges*0.6))
    for instance in range(20):
        print(instance)
        tries = 0
        max_tries = 20
        not_found = True
        while (not_found) and (tries < max_tries):
            tries += 1

            # initialize a graph
            G = igraph.Graph.Erdos_Renyi(n=num_nodes, m=int(np.floor(num_edges*1.05)), directed=False, loops=False)
            G = G.components().giant() # we want a connected graph to start with
            V = [v.index for v in G.vs()]
            E = G.get_edgelist()
            
            if len(E) > num_edges:
                degrees = igraph.GraphBase.degree(G)
                degree_1_nodes = [i for i, x in enumerate(degrees) if x == 1]
                n_edges_to_prune = len(E) - num_edges
                n_degree_1_nodes = len(degree_1_nodes)
                if n_degree_1_nodes > n_edges_to_prune:
                    G.delete_vertices(degree_1_nodes[0:n_edges_to_prune])
                    V = [v.index for v in G.vs()]
                    E = G.get_edgelist()

            if len(E) == num_edges:
                not_found = False
                if verbose:
                    print('Graph has %d nodes and %d edges in total.'%(len(V),len(E)))
                    print('Mean degree: %0.2f'%np.mean([b for b in G.degree(V)]))
                    print('---------------------------------------')
                
                # Check if the graph instance has cycles
                Gnx = nx.Graph()
                Gnx.add_nodes_from(V)
                Gnx.add_edges_from(E)
                if verbose:
                    print('The graph instance has %d cycles.'%len(nx.cycle_basis(Gnx)))

                G.write_graphml("./synthetic_graphs/er/graphs/G_n_edges_%d_instance_%d.graphml"%(len(E), instance)) # save graph                
                
                # generate costs
                cost_E = dict()
                for e in E:
                    cost = np.random.rand()
                    cost_E[e] = cost
                total_cost = sum([cost_E[i] for i in E])
                cost_E_list = [cost_E[e] for e in E]
                G_spanning_tree_edges = G.spanning_tree(return_tree = False, weights=cost_E_list)
                total_mst_cost = sum([cost_E[E[i]] for i in G_spanning_tree_edges])
                cost_data = (cost_E, total_cost, total_mst_cost)
                pickle.dump(cost_data, open("./synthetic_graphs/er/edge_costs/G_n_edges_%d_instance_%d_cost_data.p"%(len(E), instance), "wb"))

                # initialize demand between each source-destination pair
                SD = []
                for s in V:
                    for d in V:
                        SD.append((s,d))
                        SD.append((d,s))
                SD = list(set(SD))
                demand = dict.fromkeys(SD)
                for vpair in SD:
                    s = vpair[0]
                    d = vpair[1]
                    if s != d:
                        demand[(s,d)] = np.random.randint(1,200)
                        demand[(d,s)] = np.random.randint(1,200)
                    else:
                        demand[(s,d)] = 0
                        demand[(d,s)] = 0
                pickle.dump(demand, open("./synthetic_graphs/er/demands/G_n_edges_%d_instance_%d_demand_data.p"%(len(E), instance), "wb" ))
                
                # translate demand to values for each edge in isolation (used by supermodular heuristic)
                values = dict()
                for e in E:
                    values[e] = demand[(e[0], e[1])] + demand[(e[1], e[0])]
                pickle.dump(values, open("./synthetic_graphs/er/edge_values/G_n_edges_%d_instance_%d_edge_values_data.p"%(len(E), instance), "wb" ))
